[View in Colaboratory](https://colab.research.google.com/github/tjdahlke/satellite_imagery_analysis/blob/master/timeseries_movie.ipynb)

# Timeseries movie of AOI

---
##Pictures are great, but movies are much cooler.
They give us another dimension to analyze our data in. That is the main strength of Planet imagery; the high freuquency of capture for a given AOI. In the [previous notebook](https://colab.research.google.com/drive/1yFD3cvevSrtas3N457okPhz220jfUrvO), we were able to test out the Planet API to clip and download a single image for analysis and RGB reconstruction. Here, we will loop through all the results for a given date range and then display the output as a movie, showing the progression of the imagery over time. This will give us a rough idea of what kinds of trends that might be interesting to analyze, as well as what kind of problems / noise in our time series that we will have to deal with later in our workflow.

## Setup API key

In [26]:
PLANET_API_KEY='yourPlanetAPIKeyHere'
!echo $PLANET_API_KEY

yourPlanetAPIKeyHere


## Setup environment


In [6]:
!pip install tqdm
!pip install array2gif
!mkdir -p output
!ls output/
import os
outputpath=os.getcwd()+'/output'

20170818_190852_0f2b_3B_AnalyticMS_SR_clip.tif
20170906_190637_0f4a_3B_AnalyticMS_SR_clip.tif
20170910_181210_1010_3B_AnalyticMS_SR_clip.tif
20170919_181407_0f1b_3B_AnalyticMS_SR_clip.tif
20170919_181705_0e20_3B_AnalyticMS_SR_clip.tif
20170921_181400_1031_3B_AnalyticMS_SR_clip.tif
20170923_181235_0f42_3B_AnalyticMS_SR_clip.tif
20170925_181354_103e_3B_AnalyticMS_SR_clip.tif
20170927_181358_0f17_3B_AnalyticMS_SR_clip.tif
20170927_181359_0f17_3B_AnalyticMS_SR_clip.tif
20171002_181141_1033_3B_AnalyticMS_SR_clip.tif
20171002_181704_0e0f_3B_AnalyticMS_SR_clip.tif
20171006_181305_103b_3B_AnalyticMS_SR_clip.tif
20171008_190245_1_0f2e_3B_AnalyticMS_SR_clip.tif
20171014_181258_1034_3B_AnalyticMS_SR_clip.tif
20171015_181445_0f28_3B_AnalyticMS_SR_clip.tif
20171015_190143_104d_3B_AnalyticMS_SR_clip.tif
20171016_181457_0f1b_3B_AnalyticMS_SR_clip.tif
20171022_181306_1039_3B_AnalyticMS_SR_clip.tif
20171022_190017_0f2d_3B_AnalyticMS_SR_clip.tif
20171023_181352_1044_3B_AnalyticMS_SR_clip.tif
20171025_19

## Add functions to Python environment

In [0]:
import time
from tqdm import tqdm
import zipfile

def download_image(clip_download_url,local_folder):
    # Download clip
    response = requests.get(clip_download_url, stream=True)
    zipname=local_folder + scene_id + '.zip'
    with open(zipname, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)
    # Extract the image file
    ziped_item = zipfile.ZipFile(zipname,'r')
    imagename=ziped_item.namelist()[0]
    outputpath=os.getcwd()+'/'+local_folder
    zf = zipfile.ZipFile(zipname)
    zf.extract(imagename,outputpath)    
    # Delete zip file
    os.remove(local_folder + scene_id + '.zip')
    imagefile=outputpath+imagename
    print('Downloaded clip located at: %s \n' %(imagefile))
    return imagefile
  
  
# Define a new class and functions for image retrieval that use the api_key
class image_retrieval(object):
    def __init__(self, api_key):
        self.api_key=api_key
        return

    def check_assets(self,item_type,scene_id):
        # Check to see what assets exist for this item
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, scene_id)
        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
          requests.get(
            id0_url,
            auth=HTTPBasicAuth(self.api_key, '')
          )
        # List of asset types available for this particular satellite image
        return result.json().keys()
      
      
    def request_clip(self,item_type,scene_id,asset_type):
        request_succeeded=False
        post_try=1
        clip_download_url=None
        while((not request_succeeded) and (post_try<5)):
            # Construct clip API payload
            clip_payload = {
                'aoi': geojson_geometry,
                'targets': [
                  {
                    'item_id': scene_id,
                    'item_type': item_type,
                    'asset_type': asset_type
                  }
                ]
            }
            # Request clip of scene (This will take some time to complete)
            request = requests.post('https://api.planet.com/compute/ops/clips/v1', auth=(self.api_key, ''), json=clip_payload)
            # Check that the request actually ran OK
            a=str(request)
            b=a.split('<Response [')[1]
            val=b.split(']>')[0]

            if (val=='202'):
                request_succeeded=True
                clip_url = request.json()['_links']['_self']
                # Poll API to monitor clip status. Once finished, download and upzip the scene
                clip_succeeded = False
                while not clip_succeeded:
                    # Poll API
                    check_state_request = requests.get(clip_url, auth=(PLANET_API_KEY, ''))
                    # Check that the request actually ran OK
                    a=str(check_state_request)
                    b=a.split('<Response [')[1]
                    val=b.split(']>')[0]
                    if (val=='200'):
                        # If clipping process succeeded , we are done
                        if check_state_request.json()['state'] == 'succeeded':
                            clip_download_url = check_state_request.json()['_links']['results'][0]
                            clip_succeeded = True
                            print("Clip of scene succeeded and is ready to download") 
                        # Still activating. Wait 5 second and check again.
                        else:
                            print("...Still waiting for clipping to complete...")
                            time.sleep(5)
            else:
                print("Request post has failed. This is attempt # %s" % (post_try))
                request_succeeded=False
                post_try=post_try+1
        
        return clip_download_url

## Select an Area of Interest (AOI)

In [0]:
# Parking lot at Six-Flags Vallejo
geojson_geometry={
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.23056077957153,
              38.13398272942372
            ],
            [
              -122.23123669624327,
              38.13374643791151
            ],
            [
              -122.23177313804626,
              38.13374643791151
            ],
            [
              -122.23236322402953,
              38.13388146172644
            ],
            [
              -122.23273873329163,
              38.13379707187137
            ],
            [
              -122.2328782081604,
              38.1336282918685
            ],
            [
              -122.23339319229126,
              38.13283502062674
            ],
            [
              -122.2338545322418,
              38.132353990248916
            ],
            [
              -122.2341763973236,
              38.131645097597776
            ],
            [
              -122.23411202430725,
              38.13119781654914
            ],
            [
              -122.23381161689757,
              38.130919319417266
            ],
            [
              -122.23248124122618,
              38.13090244076908
            ],
            [
              -122.23114013671875,
              38.13089400144351
            ],
            [
              -122.23015308380126,
              38.13095307670197
            ],
            [
              -122.23011016845702,
              38.13134972362775
            ],
            [
              -122.23016381263733,
              38.13226959862768
            ],
            [
              -122.23023891448975,
              38.133071315089744
            ],
            [
              -122.23056077957153,
              38.13398272942372
            ]
          ]
        ]
      }

Now we set the other filter criteria such as cloud cover, date range, etc.

In [0]:
# Get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# Get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2011-08-31T00:00:00.000Z",
    "lte": "2018-02-01T00:00:00.000Z"
  }
}

# Only get images which have <10% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# Combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter,cloud_cover_filter,date_range_filter]
}

## Searching and Clipping the data
Now we send a post request via the API to retrieve the results of all images that fall within our filter criteria.

In [0]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

# API request object
search_request = {
    "interval": "day",
    "item_types": ["PSScene4Band"],
    "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# print(json.dumps(search_result.json(), indent=2)[0:3500][:])

Our search returns metadata for all of the images within our AOI that match our date range and cloud coverage filters. It looks like there are multiple images here; let's extract a list of just the image IDs:

In [11]:
# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
item_typeList = [feature['properties']['item_type'] for feature in search_result.json()['features']]

print(image_ids)

['20180128_181858_1013', '20180128_181857_1013', '20180127_184519_1020', '20180127_184518_1020', '20180126_181715_0f35', '20180120_184645_0f36', '20180119_181639_1034', '20171228_184944_0f40', '20171221_185115_103f', '20171221_185114_1_103f', '20171221_181538_0f15', '20171220_181752_0e0e', '20171218_181651_0e20', '20171217_181631_1032', '20171214_181611_103e', '20171214_181613_103e', '20171213_185244_0f44', '20171213_181603_100c', '20171212_185234_0f49', '20171212_181749_0e26', '20171210_181621_100a', '20171207_181635_101b', '20171207_181545_103c', '20171206_181621_1027', '20171204_181605_1010', '20171129_181539_1022', '20171129_181538_1022', '20171127_185544_0f29', '20171127_181533_101e', '20171127_181532_101e', '20171119_181607_0f52', '20171119_181606_0f52', '20171118_181526_1030', '20171107_181609_0f22', '20171107_181610_0f22', '20171107_181449_1035', '20171031_175901_1_0c59', '20171106_181635_101b', '20171102_181652_0e26', '20171031_181427_1015', '20171028_181459_0f25', '20171027_1

## Loop through and download a series of images
In this step we will loop over the images in the list that we just retrieved and download them.

In [12]:

asset_type = 'analytic_sr'
item_type = 'PSScene4Band'
# print(len(image_ids))
bad_scenes=[]


# Make list of all the downloaded files so we dont download twice
downloaded = []
for root, dirs, files in os.walk(outputpath):
    for file in files:
        if file.endswith('.tif'):
            downloaded.append(file)            
downloaded.sort(reverse=True)            
print(downloaded)
print("%d files downloaded" % (len(downloaded)))
  
# Download any missing files
for ii in range(0,len(image_ids)):
#     if x not in unique_list:
    scene_id = image_ids[ii]
    # Initialize image retrieval object
    imgRetObj=image_retrieval(PLANET_API_KEY)
    # Post request to download the clipped scene
    clip_download_url=imgRetObj.request_clip(item_type,scene_id,asset_type)
    if(not (clip_download_url==None)):
        # Download the file
        filename=download_image(clip_download_url,'output/')
    else:
        print("Failed for scene_id:\n %s" %(scene_id))
        bad_scenes.append(scene_id)
    time.sleep(5)


['20180128_181858_1013_3B_AnalyticMS_SR_clip.tif', '20180128_181857_1013_3B_AnalyticMS_SR_clip.tif', '20180127_184519_1020_3B_AnalyticMS_SR_clip.tif', '20180127_184518_1020_3B_AnalyticMS_SR_clip.tif', '20180126_181715_0f35_3B_AnalyticMS_SR_clip.tif', '20180120_184645_0f36_3B_AnalyticMS_SR_clip.tif', '20180119_181639_1034_3B_AnalyticMS_SR_clip.tif', '20171228_184944_0f40_3B_AnalyticMS_SR_clip.tif', '20171221_185115_103f_3B_AnalyticMS_SR_clip.tif', '20171221_185114_1_103f_3B_AnalyticMS_SR_clip.tif', '20171221_181538_0f15_3B_AnalyticMS_SR_clip.tif', '20171220_181752_0e0e_3B_AnalyticMS_SR_clip.tif', '20171218_181651_0e20_3B_AnalyticMS_SR_clip.tif', '20171217_181631_1032_3B_AnalyticMS_SR_clip.tif', '20171214_181611_103e_3B_AnalyticMS_SR_clip.tif', '20171213_185244_0f44_3B_AnalyticMS_SR_clip.tif', '20171213_181603_100c_3B_AnalyticMS_SR_clip.tif', '20171212_185234_0f49_3B_AnalyticMS_SR_clip.tif', '20171212_181749_0e26_3B_AnalyticMS_SR_clip.tif', '20171210_181621_100a_3B_AnalyticMS_SR_clip.tif

KeyboardInterrupt: ignored

## Combine time series images
Now that we've downloaded our files, we can make the RBG images of each and concatenate them together to make a movie.

In [13]:
# Make list of all the downloaded files
downloaded = []
for root, dirs, files in os.walk(outputpath):
    for file in files:
        if file.endswith('.tif'):
            downloaded.append(file)            
downloaded.sort(reverse=True)            
print(downloaded)
print("%d files downloaded" % (len(downloaded)))

['20180128_181858_1013_3B_AnalyticMS_SR_clip.tif', '20180128_181857_1013_3B_AnalyticMS_SR_clip.tif', '20180127_184519_1020_3B_AnalyticMS_SR_clip.tif', '20180127_184518_1020_3B_AnalyticMS_SR_clip.tif', '20180126_181715_0f35_3B_AnalyticMS_SR_clip.tif', '20180120_184645_0f36_3B_AnalyticMS_SR_clip.tif', '20180119_181639_1034_3B_AnalyticMS_SR_clip.tif', '20171228_184944_0f40_3B_AnalyticMS_SR_clip.tif', '20171221_185115_103f_3B_AnalyticMS_SR_clip.tif', '20171221_185114_1_103f_3B_AnalyticMS_SR_clip.tif', '20171221_181538_0f15_3B_AnalyticMS_SR_clip.tif', '20171220_181752_0e0e_3B_AnalyticMS_SR_clip.tif', '20171218_181651_0e20_3B_AnalyticMS_SR_clip.tif', '20171217_181631_1032_3B_AnalyticMS_SR_clip.tif', '20171214_181611_103e_3B_AnalyticMS_SR_clip.tif', '20171213_185244_0f44_3B_AnalyticMS_SR_clip.tif', '20171213_181603_100c_3B_AnalyticMS_SR_clip.tif', '20171212_185234_0f49_3B_AnalyticMS_SR_clip.tif', '20171212_181749_0e26_3B_AnalyticMS_SR_clip.tif', '20171210_181621_100a_3B_AnalyticMS_SR_clip.tif

Now we will concatenate them together into a GIF image that we can view.

In [14]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation as animation
import numpy as np

# Reading the downloaded image into an array
count=0
new=None
for imagefile in downloaded:
    fullpath=outputpath+'/'+imagefile
    image = cv2.imread(fullpath, -1)
    red=image[:,:,0]
    green=image[:,:,1]
    blue=image[:,:,2]
    nir=255.99*image[:,:,3]/np.amax(image[:,:,3])
    # Stack and normalize the RGB values to 256
    rgb=np.dstack((red,green,blue))
    rgb=255.99*rgb/np.amax(rgb)
    rgb_uint8 = (rgb) .astype(np.uint8)
    # Adding this image array to a concatentated array
    choice=rgb_uint8
    if(count==0):
        new=choice[np.newaxis]
    else:
        new=np.concatenate((new, choice[np.newaxis]), axis=0)
    count=count+1 
    
# Transpose the array    
newT=new.transpose()

print(newT.shape)

(3, 119, 115, 61)


In [25]:
# Build the animation
numframes=newT.shape[3]
fig = plt.figure(figsize=(8, 8))
Z   = []
img = []
count=0
threshold=75.0
for i in range(0,numframes):
    qq=newT[:,:,:,i]
    Z.append(qq.transpose())
    if(np.count_nonzero(Z[i])>threshold*3*qq.shape[1]*qq.shape[2]/100.0):
        img.append([plt.imshow(Z[i])])
        count=count+1
percent=count/numframes
print("%f percent of the frames had more than %f percent non-zero elements" % (percent,threshold))
plt.close()
# Plot the animation
ani = animation.ArtistAnimation(fig, img, interval=20, blit=True,repeat_delay=0.2)
ani

0.901639 percent of the frames had more than 75.000000 percent non-zero elements
